In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torchvision import transforms
from collections import Counter
from torchmetrics import AUROC
from statistics import mean, stdev

In [ ]:
transforms = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    torchvision.transforms.ColorJitter(hue=.05, saturation=.05, brightness = .05),
    torchvision.transforms.RandomHorizontalFlip(p = 0.5),
    torchvision.transforms.RandomVerticalFlip(p = 0.5),
    torchvision.transforms.RandomRotation(degrees = (0, 180)),
    ])

In [ ]:
train_data = torchvision.datasets.ImageFolder(root = '/mnt/d/MHIST/train', transform = transforms )
test_data = torchvision.datasets.ImageFolder(root = '/mnt/d/MHIST/test', transform = transforms)

In [ ]:
train_size = int(0.9 * len(train_data))
val_size = len(train_data) - train_size

In [ ]:
train_dataset, val_dataset = torch.utils.data.random_split(train_data, [train_size, val_size])

In [ ]:
train_dataset.dataset.class_to_idx
print(dict(Counter(train_dataset.dataset.targets)))

In [ ]:
batch_size = 32

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size= batch_size, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size= batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size= batch_size)

In [ ]:
model = torchvision.models.resnet18(pretrained = True)
model.fc = nn.Sequential(nn.Linear(512,2))
model = model.to(device = 'cuda')

In [ ]:
loss = torch.nn.CrossEntropyLoss()

In [ ]:
def train(num_epochs):
    best_accuracy = 0.0
    
    print("Starting training...")
    
    for epoch in range(1, num_epochs + 1):
        running_train_loss = 0.0 
        running_accuracy = 0.0 
        running_val_loss = 0.0 
        total = 0.0        
        # Training loop
        
        train_auroc = AUROC(pos_label = 1, num_classes = 2)
        val_auroc = AUROC(pos_label = 1, num_classes = 2)
        
        for data in train_loader:
            inputs, outputs = data
            inputs = inputs.to(device = 'cuda')
            outputs = outputs.to(device = 'cuda')
            optimizer.zero_grad()             
            predicted_outputs = model(inputs)   
            train_loss = loss(predicted_outputs, outputs)   
            train_loss.backward()   
            optimizer.step()        
            running_train_loss +=train_loss.item()
            train_auroc_batch = train_auroc(predicted_outputs, outputs)
                        
        # Calculate training loss value 
        train_loss_value = running_train_loss/len(train_loader) 
        avg_train_auroc = train_auroc.compute()
        
        # Validation Loop 
        with torch.no_grad(): 
            model.eval()
            for data in val_loader:
                inputs, outputs = data
                inputs = inputs.to(device = 'cuda')
                outputs = outputs.to(device = 'cuda')
                predicted_outputs = model(inputs)
                val_loss = loss(predicted_outputs, outputs)
                
                _, predicted = torch.max(predicted_outputs, 1)
                running_val_loss += val_loss.item()
                total += outputs.size(0)
                running_accuracy += (predicted == outputs).sum().item()
                val_auroc_batch = val_auroc(predicted_outputs, outputs)
         
                
        # Calculate validation loss value 
        val_loss_value = running_val_loss/len(val_loader) 
        avg_val_auroc = val_auroc.compute()        
        # Calculate accuracy as the number of correct pred
        accuracy = (100 * running_accuracy / total)     
        
        print('EPOCH', epoch,
              'Training Loss: %.4f' %train_loss_value, 
              'Train AUROC: %.4f' %avg_train_auroc,
              'Validation Loss: %.4f' %val_loss_value,
              'Validation AUROC: %.4f' %avg_val_auroc,
              'Accuracy %d %%' % (accuracy))
        train_auroc.reset()
        val_auroc.reset()

In [ ]:
def test(model):
    test_auroc = AUROC(pos_label = 1, num_classes = 2)
    with torch.no_grad():
        model.eval()
        for data in test_loader:
            inputs, outputs = data
            inputs = inputs.to(device = 'cuda')
            outputs = outputs.to(device = 'cuda')
            predicted_outputs = model(inputs)
            _, predicted = torch.max(predicted_outputs, 1)
            test_auroc_batch = test_auroc(predicted_outputs, outputs)
    
    avg_test_auroc = test_auroc.compute()
    return avg_test_auroc

In [ ]:
results = []

for i in range(9):
    
    seed = i
    torch.manual_seed(seed)
    
    model = torchvision.models.resnet18(pretrained = True)
    model.fc = nn.Sequential(nn.Linear(512,2))
    model = model.to(device = 'cuda')
    
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9, weight_decay = 0.01)
    train(30)
    
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.0001, momentum = 0.9, weight_decay = 0.01)
    train(10)
    
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.00001, momentum = 0.7, weight_decay = 0.01)
    train(10)
    
    results.append(test(model))

In [ ]:
results = [result.item() for result in results] 

In [ ]:
print('Mean:',"{:.3f}".format(mean(results)*100),'\nStandard deviation:', "{:.3f}".format(stdev(results)*100))